# Animations

This notebook contains animation output functions for Constitutive Models, more specifically for a bounding surface model by Borjas & Amies, 1994. The functions are developed and maintained by Justin Bonus (University of Washington).

Use ``%run YOURPATH/'Bounding Surface'/Animations.ipynb`` at the start of your notebook to import these functions.

In [5]:
def animateDeviatoricSurface(time, Stress0, stress, Su):
    # ==============================================
    # Requires access to modified drawDeviatoricSurface_function() by original author Pedro Arduino
    # Author: Justin K. Bonus, University of Washington, July 2019
    # ==============================================
    # This function animates stresses in a pseudo 2D deviatoric space for the Borjas & Amies
    # 1994 multiaxial cyclic bounding surface model. 
    # Draws bounding surface, yield surface, \kappa contours, and various axis/stresses
    # =================== Inputs ===================
    # time: List of time instances evaluated during stress loading, determines number of frames
    # Stress0: Array containing 6x1 stresses representing last unloading point
    # stress: Array containing 6x1 stresses representing the current stress
    # Su: Undrained shear strength, defines boundary radius (R=Su(8/3)^0.5)
    # =================== Output ===================
    # anidev: Animation for the stress loading in deviatoric space
    # mp4 anidev: Saves .mp4 of animation to directory
    # gif anidev: Saves .gif of animation to directory. Compression method currently disabled
    # HTML anidev: Animation embedded in HTML video format, appears in jupyter notebooks
    # ==============================================
    
    import numpy as np
    import matplotlib.pyplot as plt
    import mpl_toolkits.mplot3d.axes3d as p3
    import matplotlib.animation as animation
    from matplotlib import rc
    from IPython.display import HTML #Allows for HTML output of animation   

    fps = 10
    R = Su*(8/3)**0.5

    # Returns instance of objects from drawDeviatoricSurface_function in a namedtuple for ease of use
    def Gen_Frame(Stress0, stress, Su):
        # Create namedtuple containing all lines/circles for the frame
        drawDev = drawDeviatoricSurface_function(Stress0, stress, Su)
        return drawDev

    # Initialize wire_frames for iteration, global access for convenience
    global VMframe, Bframe
    global BKOneframe, BKTwoframe, BKThreeframe, BKFourframe
    global alphaframe, stressframe, zetaframe
    VMframe, Bframe = None, None
    BKOneframe, BKTwoframe, BKThreeframe, BKFourframe = None,None,None,None
    alphaframe, stressframe, zetaframe = None, None, None
    
    # Test Gen_Frame for first stress state
    hold = Gen_Frame(Stress0[:,0], stress[:,0], Su)

    # Update function called each frame, returns new drawings based on output of Gen_Frame
    def update(frame):
        global VMframe, Bframe
        global BKOneframe, BKTwoframe, BKThreeframe, BKFourframe
        global alphaframe, stressframe, zetaframe
        # If a line collection is already remove it before drawing. Creates a 'clean-slate' each frame
        if VMframe:
            ax.collections.remove(VMframe)
        if Bframe:
            ax.collections.remove(Bframe)
        if BKOneframe:
            ax.collections.remove(BKOneframe)
        if BKTwoframe:
            ax.collections.remove(BKTwoframe)
        if BKThreeframe:
            ax.collections.remove(BKThreeframe)
        if BKFourframe:
            ax.collections.remove(BKFourframe)
        if alphaframe:
            ax.collections.remove(alphaframe)
        if stressframe:
            ax.collections.remove(stressframe)
        if zetaframe:
            ax.collections.remove(zetaframe)
        
        # Get the namedtuple containing all wireframes for the stress state of this frame
        hold = Gen_Frame(Stress0[:,frame], stress[:,frame], Su)
        
        # Plot the new wireframes.
        alphaframe = ax.plot_wireframe(hold.dev_alphaLinex, hold.dev_alphaLiney, hold.dev_alphaLinez, 
                                       rstride=1, cstride=1, color='blue', linewidth=2)
        stressframe = ax.plot_wireframe(hold.dev_stressLinex, hold.dev_stressLiney, hold.dev_stressLinez, 
                                        rstride=1, cstride=1, color='grey', label = '$\sigma\'$', linewidth=2)
        zetaframe = ax.plot_wireframe(hold.dev_zetaLinex, hold.dev_zetaLiney, hold.dev_zetaLinez, 
                                      rstride=1, cstride=1, color='green', label = '$\zeta\'$', linewidth=2)
        VMframe = ax.plot_wireframe(hold.VMSurf[:,2], hold.VMSurf[:,1], hold.VMSurf[:,0].reshape(1,len(hold.VMSurf[:,0])), 
                                    rstride=1, cstride=1, color='black', label='Bounding Surface', linewidth=2)
        #Bframe = ax.plot_wireframe(hold.BSurf[:,2], hold.BSurf[:,1], hold.BSurf[:,0].reshape(1,len(hold.BSurf[:,0])), 
        #                           rstride=1, cstride=1, color='blue', label='Yield Surface', linewidth=2)
        BKOneframe = ax.plot_wireframe(hold.BKSurfOne[:,2], hold.BKSurfOne[:,1], hold.BKSurfOne[:,0].reshape(1,len(hold.BKSurfOne[:,0])), 
                                   rstride=1, cstride=1, color='yellow', label='kappa', linewidth=2)
        BKTwoframe = ax.plot_wireframe(hold.BKSurfTwo[:,2], hold.BKSurfTwo[:,1], hold.BKSurfTwo[:,0].reshape(1,len(hold.BKSurfTwo[:,0])), 
                                   rstride=1, cstride=1, color='orange', label='kappa', linewidth=2)
        BKThreeframe = ax.plot_wireframe(hold.BKSurfThree[:,2], hold.BKSurfThree[:,1], hold.BKSurfThree[:,0].reshape(1,len(hold.BKSurfThree[:,0])), 
                                   rstride=1, cstride=1, color='red', label='kappa', linewidth=2)
        BKFourframe = ax.plot_wireframe(hold.BKSurfFour[:,2], hold.BKSurfFour[:,1], hold.BKSurfFour[:,0].reshape(1,len(hold.BKSurfFour[:,0])), 
                                   rstride=1, cstride=1, color='purple', label='kappa', linewidth=2)

    #Settings for plot
    def init():
        fig_anidev.set_figheight(6)
        fig_anidev.set_figwidth(6)
        ax.set_xlim(-R, R)
        ax.set_ylim(-R, R)
        ax.set_zlim(-R, R)
        #Set view to deviatoric space
        #view(-45, -35.25)
        ax.azim = 45
        ax.elev = 35.25
        #view([-0.5773 -0.5773 -0.5773])
        #plt.legend(bbox_to_anchor=(.75,1), loc="lower right")  # Legend outside plot
        #Make the panes transparent
        ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
        ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
        ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

        #Make the grid lines transparent
        ax.set_axis_off()
        ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
        ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
        ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
        
        #Draw constant objects
        h = Gen_Frame(Stress0[:,0],stress[:,0],Su)
        ax.plot_wireframe(h.refLine1x,h.refLine1y,h.refLine1z,color='black')
        ax.plot_wireframe(h.refLine2x,h.refLine2y,h.refLine2z,color='black')
        ax.plot_wireframe(h.refLine3x,h.refLine3y,h.refLine3z,color='black')
        ax.legend()
        ax.set_title('Stress Loading Viewed from Deviatoric Space')


    #Attaching 3D axis to the figure
    fig_anidev = plt.figure(figsize=(6,6))
    fig_anidev.set_size_inches(w=6,h=6)
    ax = p3.Axes3D(fig_anidev)
    
    #Using the fig_dev plotting space FuncAnimation() calls the draw() function. len(stress) is passed into draw()
    #first and in this case controls number of frames. fargs passes in additional draw() parameters. interval
    #determines millisecond delay between frames. blit 
    anidev = animation.FuncAnimation(fig_anidev, update, len(time)-1, init_func=init, interval=1000/fps, blit=False, repeat=True)
    plt.close();
    plt.ioff()
    
    #Save .gif and .mp4 of animation in directory, increases compile time. Toggle off for speed
    fn = 'deviatoricSurface_funcanimation'
    ##anidev.save(fn+'.mp4',writer='ffmpeg',fps=fps)
    ##anidev.save(fn+'.gif',writer='imagemagick',fps=fps)

    #import subprocess
    #cmd = 'magick convert %s.gif -fuzz 5%% -layers Optimize %s_r.gif'%(fn,fn)
    #subprocess.check_output(cmd, shell=True)

    #Embed HTML video of animation in notebook
    #HTML(anidev.to_html5_video())
    #rc('animation', html='html5')
    plt.rcParams['animation.html'] = 'html5'
    return anidev

In [3]:
from matplotlib import animation, rc
from matplotlib.lines import Line2D
from IPython.display import HTML #Allows for HTML output of animation  

class quadSubplotAnimation(animation.TimedAnimation):
    #===============================================================================================
    #This class allows easy creation of animated 2D (2x2) subplots in embedded HTML format
    #User must specify x, xlabel, y, ylabel for each of the 4 plots before calling the class
    #Requires ffmpeg for encoding
    #===============================================================================================
    import numpy as np
    #%matplotlib notebook
    import matplotlib.pyplot as plt
    

    def __init__(self):
        fig = plt.figure()
        ax1 = fig.add_subplot(2, 2, 1)
        ax2 = fig.add_subplot(2, 2, 2)
        ax3 = fig.add_subplot(2, 2, 3)
        ax4 = fig.add_subplot(2, 2, 4)
        plt.tight_layout()
        #ax4 = fig.add_subplot(2, 2)
        
        #Define the variables on the right before calling this class
        self.t = out.time
        self.x1, self.xlabel1, self.y1, self.ylabel1 = x1, xlabel1, y1, ylabel1
        self.x2, self.xlabel2, self.y2, self.ylabel2 = x2, xlabel2, y2, ylabel2
        self.x3, self.xlabel3, self.y3, self.ylabel3 = x3, xlabel3, y3, ylabel3
        self.x4, self.xlabel4, self.y4, self.ylabel4 = x4, xlabel4, y4, ylabel4

        ax1.set_xlabel(self.xlabel1)
        ax1.set_ylabel(self.ylabel1)
        self.line1 = Line2D([], [], color='black')
        self.line1a = Line2D([], [], color='red', linewidth=2)
        self.line1e = Line2D(
            [], [], color='red', marker='o', markeredgecolor='r')
        ax1.add_line(self.line1)
        ax1.add_line(self.line1a)
        ax1.add_line(self.line1e)
        ax1.set_xlim(x1.min(), x1.max())
        ax1.set_ylim(y1.min(), y1.max())
        #ax1.set_aspect('equal', 'datalim')

        ax2.set_xlabel(self.xlabel2)
        ax2.set_ylabel(self.ylabel2)
        self.line2 = Line2D([], [], color='black')
        self.line2a = Line2D([], [], color='red', linewidth=2)
        self.line2e = Line2D(
            [], [], color='red', marker='o', markeredgecolor='r')
        ax2.add_line(self.line2)
        ax2.add_line(self.line2a)
        ax2.add_line(self.line2e)
        ax2.set_xlim(x2.min(), x2.max())
        ax2.set_ylim(y2.min(), y2.max())

        ax3.set_xlabel(self.xlabel3)
        ax3.set_ylabel(self.ylabel3)
        self.line3 = Line2D([], [], color='black')
        self.line3a = Line2D([], [], color='red', linewidth=2)
        self.line3e = Line2D(
            [], [], color='red', marker='o', markeredgecolor='r')
        ax3.add_line(self.line3)
        ax3.add_line(self.line3a)
        ax3.add_line(self.line3e)
        ax3.set_xlim(x3.min(), x3.max())
        ax3.set_ylim(y3.min(), y3.max())

        ax4.set_xlabel(self.xlabel4)
        ax4.set_ylabel(self.ylabel4)
        self.line4 = Line2D([], [], color='black')
        self.line4a = Line2D([], [], color='red', linewidth=2)
        self.line4e = Line2D(
            [], [], color='red', marker='o', markeredgecolor='r')
        ax4.add_line(self.line4)
        ax4.add_line(self.line4a)
        ax4.add_line(self.line4e)
        ax4.set_xlim(x4.min(), x4.max())
        ax4.set_ylim(y4.min(), y4.max())
        fig.set_size_inches(h=8,w=8)
        plt.tight_layout()
        animation.TimedAnimation.__init__(self, fig, interval=50, blit=True)

    def _draw_frame(self, framedata):
        i = framedata
        head = i - 1
        head_slice = (self.t > self.t[i] - 1.0) & (self.t < self.t[i])

        self.line1.set_data(self.x1[:i], self.y1[:i])
        self.line1a.set_data(self.x1[head_slice], self.y1[head_slice])
        self.line1e.set_data(self.x1[head], self.y1[head])

        self.line2.set_data(self.x2[:i], self.y2[:i])
        self.line2a.set_data(self.x2[head_slice], self.y2[head_slice])
        self.line2e.set_data(self.x2[head], self.y2[head])

        self.line3.set_data(self.x3[:i], self.y3[:i])
        self.line3a.set_data(self.x3[head_slice], self.y3[head_slice])
        self.line3e.set_data(self.x3[head], self.y3[head])
        
        self.line4.set_data(self.x4[:i], self.y4[:i])
        self.line4a.set_data(self.x4[head_slice], self.y4[head_slice])
        self.line4e.set_data(self.x4[head], self.y4[head])

        self._drawn_artists = [self.line1, self.line1a, self.line1e,
                               self.line2, self.line2a, self.line2e,
                               self.line3, self.line3a, self.line3e,
                               self.line4, self.line4a, self.line4e]

    def new_frame_seq(self):
        return iter(range(self.t.size))

    def _init_draw(self):
        lines = [self.line1, self.line1a, self.line1e,
                 self.line2, self.line2a, self.line2e,
                 self.line3, self.line3a, self.line3e,
                 self.line4, self.line4a, self.line4e]
        for l in lines:
            l.set_data([], [])
            
    plt.close();
    plt.ioff()
    #===============================================================================================

In [1]:
def kappaMatrix_animation(kappaM):
    #================= PLOTTING KAPPA MATRIX - Animation ====================
    # Author: Justin Bonus, July 2019, University of Washington
    # 
    # This functions creates an animation for the desired size of a kappa 
    # matrix by directly calling the kappaMatrix() function. the animation
    # can be embedded as an html, saved as a .gif, or put into .mp4 format
    #========================================================================
    
    import numpy as np
    %matplotlib inline
    import matplotlib.pyplot as plt
    import mpl_toolkits.mplot3d.axes3d as p3
    import matplotlib.animation as animation
    from matplotlib import rc
    from IPython.display import HTML #Allows for HTML output of animation   

    cen_x = int(np.floor(res_x/2)) #For res_x = 11, cen_x will equal 5. This centers it relative to zero indexing
    cen_y = int(np.floor(res_y/2))
 
    kappa_matrix = kappaM
    centerline = np.linspace(0,res_y,res_y)
    extent = [0,res_x,res_y,0]
    
    fig, (ax1,ax2) = plt.subplots(1,2, sharey=True, gridspec_kw={'wspace': 0})
    
    global kappa_display
    global kappa_line
    kappa_display, kappa_line = None, None
    
    def update(frame):
        global kappa_display, kappa_line
        # If a line collection is already remove it before drawing. Creates a 'clean-slate' each frame
        #print(frame)
        if kappa_line:
            ax1.clear()
            ax2.clear()
        centerline = np.linspace(0,res_y,res_y)
        cart = kappa_matrix[:,:,frame]
        kappaline = cart[:,cen_x]
        kappa_display = ax1.imshow(cart,cmap='inferno',interpolation='nearest', norm=matplotlib.colors.LogNorm(), extent=extent, animated=True)
        ax1.set_xlabel('X-Index',fontsize=14)
        ax1.set_ylabel('Y-Index',fontsize=14)
        if frame ==0:
            plt.colorbar(kappa_display)
            fig.set_size_inches(w=10,h=5)
        kappa_line = ax2.plot(kappaline, centerline, color = 'black', antialiased=True)
        #Plot \kappa heatmap in deviatoric index view
        #extent = [0,res_x,res_y,0]
        #ax1.get_shared_y_axes().join(ax1, ax2)
      

        #Plot centerline \kappa distribution in logspace
        ax2.set_xlabel('Centerline $\kappa$ Value',fontsize=14)
        ax2.set_xscale('log')
        ax2.set_xlim([minKappa,maxKappa])
        ax2.set_ylim([res_y,0])

        # fit subplots and save fig
        #fig.tight_layout()
        #fig.set_size_inches(w=10,h=5)
        #fig_name = 'kappa_layer_visual'
        #ig.savefig(fig_name)
    
        
        
    def init():
        #Plot \kappa heatmap in deviatoric index view
        #extent = [0,res_x,res_y,0]
        #ax1.set_xlabel('X-Index',fontsize=14)
        #ax1.set_ylabel('Y-Index',fontsize=14)
        #plt.colorbar(kappa_display)

        #Plot centerline \kappa distribution in logspace
        #ax2.set_xlabel('Centerline $\kappa$ Value',fontsize=14)
        #ax2.set_xscale('log')
        #ax2.set_xlim([minKappa,maxKappa])
        #ax2.set_ylim([res_y,0])

        # fit subplots and save fig
        #fig.tight_layout()
        #fig.set_size_inches(w=10,h=5)
        #fig_name = 'kappa_layer_visual'
        #ig.savefig(fig_name)
        pass
    
    anikappa = animation.FuncAnimation(fig, update, cen_x, init_func=init, interval=150, blit=False, repeat=True)
    
    #Save .gif and .mp4 of animation in directory, increases compile time. Toggle off for speed
    #fn = 'deviatoricSurface_funcanimation' #Filename
    ##anidev.save(fn+'.mp4',writer='ffmpeg',fps=fps)
    ##anidev.save(fn+'.gif',writer='imagemagick',fps=fps)
    
    #For compressing the gif:
    #import subprocess
    #cmd = 'magick convert %s.gif -fuzz 5%% -layers Optimize %s_r.gif'%(fn,fn)
    #subprocess.check_output(cmd, shell=True)
    
    #Embed HTML video of animation in notebook
    #HTML(anidev.to_html5_video())
    #rc('animation', html='html5')
    plt.rcParams['animation.html'] = 'html5'
    return anikappa
    #===================================================================